In [1]:
from tqdm.notebook import tqdm
import os
import shutil
from pathlib import Path

import pandas as pd
import supervision as sv
import torch
from sklearn.model_selection import train_test_split

In [2]:
# Пути для сохранения разделенных выборок
images_text_segmenter_train_dir = "../../data/processed/4 Segmenter test/text_segmenter/train/images"
labels_text_segmenter_train_dir = "../../data/processed/4 Segmenter test/text_segmenter/train/labels"

images_text_segmenter_valid_dir = "../../data/processed/4 Segmenter test/text_segmenter/valid/images"
labels_text_segmenter_valid_dir = "../../data/processed/4 Segmenter test/text_segmenter/valid/labels"

images_text_segmenter_test_dir = "../../data/processed/4 Segmenter test/text_segmenter/test/images"
labels_text_segmenter_test_dir = "../../data/processed/4 Segmenter test/text_segmenter/test/labels"

images_text_recognizer_train_dir = "../../data/processed/4 Segmenter test/text_recognizer/train"

images_text_recognizer_valid_dir = "../../data/processed/4 Segmenter test/text_recognizer/valid"

images_text_recognizer_test_dir = "../../data/processed/4 Segmenter test/text_recognizer/test"

csv_root_path = "../../data/processed/4 Segmenter test"

# Создание каталогов для train, valid, test и images
os.makedirs(images_text_segmenter_train_dir, exist_ok=True)
os.makedirs(labels_text_segmenter_train_dir, exist_ok=True)

os.makedirs(images_text_segmenter_valid_dir, exist_ok=True)
os.makedirs(labels_text_segmenter_valid_dir, exist_ok=True)

os.makedirs(images_text_segmenter_test_dir, exist_ok=True)
os.makedirs(labels_text_segmenter_test_dir, exist_ok=True)

os.makedirs(images_text_recognizer_train_dir, exist_ok=True)
os.makedirs(images_text_recognizer_valid_dir, exist_ok=True)
os.makedirs(images_text_recognizer_test_dir , exist_ok=True)


############################################################################################
############# Формируем датасет из каталога Победоносцев (разметка сегментами) #############
############################################################################################


image_dir = "../../data/raw/Распознавание текстов/Победоносцев/images"
annotation_file = "../../data/raw/Распознавание текстов/Победоносцев/project-14-at-2024-03-18-16-02-b43f1e84.json"
segment_annotations = pd.read_json(annotation_file)

segment_images = []

# Перебор всех файлов изображений в image_dir
for root, dirs, files in os.walk(image_dir):
    for file in files:
        image_path = os.path.join(root, file)
        segment_images.append(image_path)
        
train_images, test_images = train_test_split(segment_images, test_size=0.2, random_state=42)
train_images, valid_images = train_test_split(train_images, test_size=0.2, random_state=42)

In [3]:
print(
    f"Размер обучающей выборки: {len(train_images):>6} \n"
    f"Размер валидационной выборки: {len(valid_images)} \n"
    f"Размер тестовой выборки: {len(test_images):>7}" 
)

Размер обучающей выборки:     32 
Размер валидационной выборки: 8 
Размер тестовой выборки:      10


In [4]:
segment_annotations.head()

,id,annotations,file_upload,drafts,predictions,data,meta,created_at,updated_at,inner_id,total_annotations,cancelled_annotations,total_predictions,comment_count,unresolved_comment_count,last_comment_updated_at,project,updated_by,comment_authors
0,664,"[{'id': 6, 'completed_by': 5, 'result': [{'id'...",d7a9001e-9.png,[],[],{'ocr': '/data/upload/14/d7a9001e-9.png'},{},2024-02-21 09:56:19.858484+00:00,2024-03-18 15:02:18.286553+00:00,1,1,0,0,0,0,NaT,14,1,[]
1,665,"[{'id': 7, 'completed_by': 5, 'result': [{'id'...",29197f5a-10.png,"[{'id': 582, 'user': 'prosvetov-ry@ranepa.ru',...",[],{'ocr': '/data/upload/14/29197f5a-10.png'},{},2024-02-21 19:58:51.822014+00:00,2024-02-21 22:00:22.058720+00:00,2,1,0,0,0,0,NaT,14,5,[]
2,666,"[{'id': 8, 'completed_by': 5, 'result': [{'id'...",3ba9de09-12.png,[],[],{'ocr': '/data/upload/14/3ba9de09-12.png'},{},2024-02-21 22:03:37.750545+00:00,2024-02-22 20:47:15.350127+00:00,3,1,0,0,0,0,NaT,14,5,[]
3,667,"[{'id': 10, 'completed_by': 5, 'result': [{'id...",6ec3d82d-14.png,"[{'id': 351, 'user': 'erpaison@gmail.com', 'cr...",[],{'ocr': '/data/upload/14/6ec3d82d-14.png'},{},2024-02-22 09:44:41.515473+00:00,2024-02-23 11:42:25.663546+00:00,4,1,0,0,0,0,NaT,14,5,[]
4,668,"[{'id': 9, 'completed_by': 5, 'result': [{'id'...",080951f6-13.png,[],[],{'ocr': '/data/upload/14/080951f6-13.png'},{},2024-02-22 09:44:41.515511+00:00,2024-02-27 10:53:23.952357+00:00,5,1,0,0,0,0,NaT,14,5,[]


In [5]:
# Получаем список ID с лейблом "Перечеркнутый текст", чтобы не включать в обучение т.к. слишком мало данных (всего 5 строк)

id_to_skip = []

for row_id, annotations in segment_annotations[["annotations"]].iterrows():
    for annotations_file in annotations.values[0]:
        for annotation in annotations_file["result"]:
            annotation_id = annotation["id"]
            if annotation["type"] == "labels":
                if annotation["value"]["labels"][0] == "Перечеркнутый текст":
                    id_to_skip.append(annotation_id)

In [6]:
# Формируем датасет в pandas

text = []
points = []
orig_points = []
images = []

for row_id, (annotations, image_name) in segment_annotations[["annotations", "file_upload"]].iterrows():
    for annotations_file in annotations:
        for annotation in annotations_file["result"]:
            # Пропускаем разметку с перечеркнутым текстом
            if annotation["id"] in id_to_skip:
                continue
            if annotation["type"] == "textarea":
                # сохраняем текст разметки
                text.append(annotation["value"]["text"][0])
                # нормируем и сохраняем координаты маски
                width = annotation["original_width"]
                height = annotation["original_height"]
                norm_points = []
                yolo_points = []
                for non_norm_points in annotation["value"]["points"]:
                    norm_points.append([
                        (non_norm_points[1] * height) / 100,
                        (non_norm_points[0] * width) / 100
                    ])
                    yolo_points.append([
                        non_norm_points[0] / 100,
                        non_norm_points[1] / 100
                    ])

                points.append(norm_points)
                orig_points.append(yolo_points)
                images.append(image_name)


In [7]:
data = pd.DataFrame(data={
    "text": text,
    "points": points,
    "orig_points": orig_points,
    "image": images
})

In [8]:
yolo_data = data.groupby("image").agg({"points": lambda x: x, "orig_points": lambda x: x, "text": lambda x: x})
yolo_data = yolo_data.reset_index()

In [9]:
yolo_data.head()

,image,points,orig_points,text
0,080951f6-13.png,"[[[378.87702886216886, 1443.0718484651702], [3...","[[[0.46237483129290935, 0.08274230811578268], ...","[1863, 4 Янв. Москва. Возобновляю лекцiи въ Ун..."
1,0a32897d-0109.jpeg,"[[[217.51314388450317, 175.51117114618694], [1...","[[[0.07246538858224069, 0.07398406254574937], ...",[14. Письмо отъ Ел. Львовны. Все предупредили ...
2,0a7607c2-19.png,"[[[226.07670908416145, 598.6752800548936], [20...","[[[0.19157608961756595, 0.04937250689761115], ...",[5 Сент. – 11 Сент. въ Москве и въ Рязани. Бед...
3,10a09237-0191.jpeg,"[[[224.4213091763455, 163.48938460181995], [19...","[[[0.06820583421018771, 0.07664662198645679], ...","[взглядывала на меня – и я не зналъ о чемъ ея,..."
4,17237b4c-0204.jpeg,"[[[259.5168938084492, 379.9423949136171], [244...","[[[0.15314082826022454, 0.08951945284872342], ...","[Твоя! О Боже, Боже! скажи мне – моя или, Твоя..."


In [10]:
import cv2
import numpy as np


def crop_and_save_polygon(image_path, out_path, polygon_coords, bbox=None):
    # Загружаем изображение
    image = cv2.imread(image_path)
    
    # Определяем средний цвет изображения для заливки
    mean_color = np.array(cv2.mean(image)).astype(int)[:3]
    
    # Возвращаем на место перепутанные местами координаты yx->xy
    res = []
    for y, x in polygon_coords:
        res.append([x, y])
    
    # Преобразуем координаты полигона в формат, понятный OpenCV
    pts = np.array(res, np.int32)
    pts = pts.reshape((-1,1,2))
    
    # Создаем маску, которая соответствует области полигона
    mask = np.zeros(image.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], 0, 255, -1)
    
    # Вырезаем область изображения, соответствующую маске
    result = cv2.bitwise_and(image, image, mask=mask)
    
    # Заливаем чёрные области тем цветом, который был подан на вход функции
    result[mask==0] = mean_color
    
    # Если задан bbox, обрежем изображение по этим координатам
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        result = result[y1:y2, x1:x2]
    
    cv2.imwrite(out_path, result)
    

def convert_mask_to_xy_bbox(mask):
    # Конвертируем полигон в bbox
    bounding_boxes = sv.polygon_to_xyxy(mask).astype(int)
    # Возвращаем на место перепутанные местами координаты yx->xy
    res_bbox = [bounding_boxes[1], bounding_boxes[0], bounding_boxes[3], bounding_boxes[2]]
    return res_bbox

In [11]:
for data_images, (images_data_dir, labels_data_dir), recognizer_images_path, csv_name in tqdm(zip(
        [train_images, valid_images, test_images], 
        [
            [images_text_segmenter_train_dir, labels_text_segmenter_train_dir],
            [images_text_segmenter_valid_dir, labels_text_segmenter_valid_dir],
            [images_text_segmenter_test_dir, labels_text_segmenter_test_dir]
        ],
        [images_text_recognizer_train_dir, images_text_recognizer_valid_dir, images_text_recognizer_test_dir],
        ["train.csv", "valid.csv", "test.csv"]
), total=3):  
    
    objects = []
    
    for image in data_images:
        image_path = Path(image)
        image_name = image_path.name
        image_suffix = image_path.suffix
        
        # # # # # # # # # # # # # # # # # # # # # # # # # # # 
        # # # # # # #   Датасет для сегментации   # # # # # #
        # # # # # # # # # # # # # # # # # # # # # # # # # # #
        
        skip_ocr = False
        
        # копируем изображение
        shutil.copy(image, images_data_dir)
        image_label_path = Path(labels_data_dir) / Path(image_name.replace(image_suffix, ".txt"))
        # создаём файл с разметкой этого изображения
        with open(image_label_path, "w") as file:
            try:
                for line in yolo_data[yolo_data["image"] == image_name]["orig_points"].values[0].tolist():
                    file.write("0 " + ' '.join(f"{item[0]} {item[1]}" for item in line) + "\n")
            # Если попалось изображение без текста
            except IndexError:
                # Переходим к след итерации, т.о. файл разметки без полигонов будет создан,
                # а разметка для OCR - нет, что нам и нужно
                skip_ocr = True
                continue
                
        # # # # # # # # # # # # # # # # # # # # # # # # # # # #  
        # # # # # # #   Датасет для распознавания   # # # # # #
        # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
        
        if not skip_ocr:
            ocr_masks, texts = [*yolo_data[yolo_data["image"] == image_name][["points", "text"]].values[0].tolist()]
            for num_mask, [mask, text] in enumerate(zip(ocr_masks, texts)):
                bbox = convert_mask_to_xy_bbox(mask)
                filename = Path(f"{num_mask}_{Path(image).name}")
                out_path = Path(recognizer_images_path) / filename
                crop_and_save_polygon(image, str(out_path), mask, bbox)
                
                objects.append({"file_name": filename, "text": text})
    
    ocr_data = pd.DataFrame(objects)
    ocr_data.to_csv(f"{csv_root_path}/{csv_name}", index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
ocr_data

,file_name,text
0,0_909ccbb0-18.png,Начало общества у В.К. Константина. Отделъ Общ...
1,1_909ccbb0-18.png,Валуевъ – М-ръ Госуд. имуществъ.
2,2_909ccbb0-18.png,20 мая. Поездка съ Катей и Соничкой черезъ Москву
3,3_909ccbb0-18.png,въ Смоленскъ. у а. в. шевандиной и у Дiодора
4,4_909ccbb0-18.png,въ Александровскомъ. вернулись 1 Iюня.
...,...,...
320,37_c6d63ae5-15.png,"Варшаву и Берлинъ, и Парижъ и Лондонъ,"
321,38_c6d63ae5-15.png,на о-въ Вайтъ. – Шенклинъ. На обратномъ
322,39_c6d63ae5-15.png,Пути черезъ Ломжу – возвращаемся
323,40_c6d63ae5-15.png,1 Сентября.
